In [ ]:
import os
import datetime
from tools import us_plots
from tools import indices
from tools import ficc
from tools import top_movers
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.platypus import Table, Paragraph, Spacer, TableStyle, BaseDocTemplate, Frame, PageTemplate
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.platypus.flowables import Image, Spacer, PageBreak
from reportlab.platypus.paragraph import Paragraph
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

In [ ]:
COMPANY_NAME = "LOREM IPSUM<br />COMPANY" # Use <br /> to break lines
AUTHOR = "HGNX"
AUTHOR_EMAIL = "contact@hgk.im"
DATE = datetime.datetime.now().strftime("%Y-%m-%d")
DATE_FILENAME = datetime.datetime.now().strftime("%Y%m%d")
# FILE_NAME = f"output_{DATE_FILENAME}.pdf"
FILE_NAME = "output.pdf"

In [ ]:
class MyDocTemplate(BaseDocTemplate):
    def __init__(self, filename, **kwargs):
        BaseDocTemplate.__init__(self, filename, **kwargs)
        frame_x = (A4[0] - 6*inch) / 2
        template = PageTemplate('normal', [Frame(frame_x, 1*inch, 6*inch, 10*inch)])
        self.addPageTemplates(template)

    def afterFlowable(self, flowable):
        "Registers TOC entries."
        if flowable.__class__ == Paragraph:
            text = flowable.getPlainText()
            self.notify('TOCEntry', (text, self.page))

In [ ]:
assets_folder = os.path.join(os.getcwd(), "assets")
font = os.path.join(assets_folder, "NotoSansKR[wght].ttf")
pdfmetrics.registerFont(TTFont('NotoSansKR', font))

doc = MyDocTemplate(FILE_NAME, pagesize=A4)

In [ ]:
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='Center', alignment=1, fontSize=14, spaceAfter=20, textColor=colors.black, fontWeight='bold'))

left_style = ParagraphStyle(name='Left', parent=styles['Normal'], alignment=0)
center_style = ParagraphStyle(name='Center', parent=styles['Normal'], alignment=1, fontSize=14, spaceAfter=20, textColor=colors.black, fontWeight='bold', leading=18)
right_style = ParagraphStyle(name='Right', parent=styles['Normal'], alignment=2, fontSize=8, leading=10)

header = [
    [
        Paragraph(COMPANY_NAME, left_style),
        "", 
        Paragraph("<b>STOCK MARKET<br />DAILY REPORT</b>", center_style),
        "", 
        Paragraph(f"{AUTHOR}<br />{AUTHOR_EMAIL}<br />{DATE}", right_style)
    ]
]

header_table_style = TableStyle([
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 20),
    ('LINEBELOW', (0, 0), (-1, 0), 1, colors.grey),
])

header_table = Table(header, colWidths=[1.5*inch, 0.5*inch, 3*inch, 0.5*inch, 1.5*inch])
header_table.setStyle(header_table_style)

story = [header_table, Spacer(1, 0.3*inch)]

In [ ]:
caption_style = ParagraphStyle(name='Caption', parent=styles['Normal'], fontSize=7, alignment=0)
caption_style2 = ParagraphStyle(name='Caption', parent=styles['Normal'], fontSize=7, alignment=0, leading=5)
caption_indices = Paragraph("● Global Stock Market", caption_style)
caption_ficc = Paragraph("● FICC", caption_style)
caption_gainers = Paragraph("● Top Gainers", caption_style)
caption_losers = Paragraph("● Top Losers", caption_style)
caption_events = Paragraph("● Key Events Today", caption_style2)

In [ ]:
table_indices_data = indices.get_all_indices_data()
table_ficc_data = ficc.get_all_ficc_data()

In [ ]:
table_gainers_data, table_losers_data = top_movers.get_top_movers_data()

In [ ]:
table_events_data = [
    ["Time", "Currency", "Event", "Forecast", "Previous"],
    ["12:15", "EUR", "Deposit Facility Rate (Jul)", "3.75%", "3.50%"],
    ["12:15", "EUR", "ECB Interest Rate Decision (Jul)", "4.25%", "4.00%"],
    ["12:30", "USD", "Core Durable Goods Orders (MoM) (Jun)", "0.0%", "0.6%"],
    ["12:30", "USD", "GDP (QoQ) (Q2)", "1.8%", "2.0%"],
    ["12:30", "USD", "Initial Jobless Claims", "235K", "228K"],
    ["12:45", "EUR", "ECB Press Conference", "-", "-"],
    ["14:00", "USD", "Pending Home Sales (MoM) (Jun)", "-0.5%", "-2.7%"]
]

In [ ]:
table_style_overview = [
    ('LINEABOVE', (0, 1), (-1, 1), 0.5, colors.grey),
    ('BOX', (0, 0), (-1, -1), 0, colors.transparent),
    ('GRID', (0, 0), (-1, -1), 0, colors.transparent),
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ('ALIGN', (0, 0), (-1, 0), 'RIGHT'),
    ('ALIGN', (0, 0), (0, -1), 'LEFT'),
    ('ALIGN', (1, 1), (-1, -1), 'RIGHT'),
    ('FONTNAME', (0, 0), (-1, -1), 'NotoSansKR'),
    ('FONTSIZE', (0, 0), (-1, -1), 5),
    ('LEFTPADDING', (0, 0), (-1, -1), 1),
    ('RIGHTPADDING', (0, 0), (-1, -1), 1),
]

table_style_movers = [
    ('LINEABOVE', (0, 1), (-1, 1), 0.5, colors.grey),
    ('BOX', (0, 0), (-1, -1), 0, colors.transparent),
    ('GRID', (0, 0), (-1, -1), 0, colors.transparent),
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ('ALIGN', (0, 0), (-1, 0), 'RIGHT'),
    ('ALIGN', (0, 0), (1, -1), 'LEFT'),
    ('ALIGN', (2, 1), (-1, -1), 'RIGHT'),
    ('FONTNAME', (0, 0), (-1, -1), 'NotoSansKR'),
    ('FONTSIZE', (0, 0), (-1, -1), 5),
    ('LEFTPADDING', (0, 0), (-1, -1), 1),
    ('RIGHTPADDING', (0, 0), (-1, -1), 1),
]

table_style_events = [
    ('LINEABOVE', (0, 3), (-1, 3), 0.5, colors.grey),
    ('TOPPADDING', (0, 0), (-1, 1), 0),
    ('BOTTOMPADDING', (0, 0), (-1, 1), 0),
    ('BOX', (0, 0), (-1, -1), 0, colors.transparent),
    ('GRID', (0, 0), (-1, -1), 0, colors.transparent),
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ('ALIGN', (0, 0), (-1, 0), 'RIGHT'),
    ('ALIGN', (0, 0), (2, -1), 'LEFT'),
    ('ALIGN', (3, 1), (-1, -1), 'RIGHT'),
    ('FONTNAME', (0, 0), (-1, -1), 'NotoSansKR'),
    ('FONTSIZE', (0, 0), (-1, -1), 5),
    ('LEFTPADDING', (0, 0), (-1, -1), 1),
    ('RIGHTPADDING', (0, 0), (-1, -1), 1),
]

table_news_style = TableStyle([
    ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('TOPPADDING', (0, 0), (-1, -1), 0),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 5)
])

In [ ]:
additional_space = 0.3*inch

total_table_width = 6*inch - 0.1*inch - additional_space
single_table_width = total_table_width / 2
column_width = single_table_width / 6
table_row_height = 0.15*inch

In [ ]:
table_overview_column_width = single_table_width / 6
table_indices = Table(table_indices_data, [column_width for _ in range(7)], [table_row_height for _ in range(len(table_indices_data))])
table_indices.setStyle(table_style_overview)

table_ficc = Table(table_ficc_data, [column_width for _ in range(7)], [table_row_height for _ in range(len(table_ficc_data))])
table_ficc.setStyle(table_style_overview)

table_overview_data = [[caption_indices, Spacer(0.1*inch, 0), caption_ficc],
                       [table_indices, Spacer(0.1*inch, 0), table_ficc]]
table_overview = Table(table_overview_data)

In [ ]:
mover_first_column_width = column_width * 0.9
mover_second_column_width = column_width + (column_width * 1.5)
mover_other_columns_width = column_width * 1.2

table_mover_width = [mover_first_column_width, mover_second_column_width] + [mover_other_columns_width for _ in range(3)]
table_gainers = Table(table_gainers_data, table_mover_width, [table_row_height for _ in range(len(table_gainers_data))])
table_gainers.setStyle(table_style_movers)

table_losers = Table(table_losers_data, table_mover_width, [table_row_height for _ in range(len(table_losers_data))])
table_losers.setStyle(table_style_movers)

table_mover_data = [[caption_gainers, Spacer(0.1*inch, 0), caption_losers],
                    [table_gainers, Spacer(0.1*inch, 0), table_losers]]
table_mover = Table(table_mover_data)

In [ ]:
table_events_width = sum(table_mover_width) * 2.133
events_column_width = table_events_width / 6
events_third_width = table_events_width - 4* events_column_width

event_table_widths = [events_column_width, events_column_width, events_third_width, events_column_width, events_column_width]
table_events_data = [[caption_events], [Spacer(0.1*inch, 0)], *table_events_data]
table_events = Table(table_events_data, event_table_widths, [table_row_height for _ in range(len(table_events_data))])
table_events.setStyle(table_style_events)

In [ ]:
us_plots.create_plots()

nasdaq_graph = Image(os.path.join(assets_folder, "nasdaq.png"), 2.4*inch, 1.6*inch)
sp500_graph = Image(os.path.join(assets_folder, "sp500.png"), 2.4*inch, 1.6*inch)
sectors_graph = Image(os.path.join(assets_folder, "sectors.png"), 2.4*inch, 1.6*inch)  # Increased the height

table_graph_data = [
    nasdaq_graph, 
    Spacer(0.1*inch, 0), 
    sp500_graph, 
    Spacer(0.1*inch, 0), 
    sectors_graph
]
table_graph = Table([table_graph_data], colWidths=[2.4*inch, 0.001*inch, 2.4*inch, 0.001*inch, 2.4*inch])

In [ ]:
def add_news_items(news_items):
    result = []
    for item in news_items:
        title, url = item
        result += [
            [
                Paragraph(f"&nbsp;&nbsp;&nbsp;&nbsp;{title}<br />&nbsp;&nbsp;&nbsp;&nbsp;<font color=blue>{url}</font>", 
                          ParagraphStyle('Left', parent=styles['Normal'], alignment=0, fontName='NotoSansKR')),
            ],
            [
                Spacer(1, 0.2*inch)
            ]
        ]
    return result

def add_news_source(source, news_items):
    result = [
        [
            Paragraph(f"<b>● {source}</b>", styles['BodyText']), 
        ]
    ]
    result += add_news_items(news_items)
    return result

In [ ]:
news = []

reuters_news = [("Dwindling excess savings could scupper markets' soft-landing hopes", "https://bit.ly/test"),
                ("Philippines need not match Fed's 25-basis point hike, finance minister says", "https://bit.ly/test"),
                ("Endgame for Fed's tightening cycle challenged by easing financial conditions", "https://bit.ly/test"),
                ("IMF, Argentina reach staff deal on loan reviews to unlock $7.5 billion", "https://bit.ly/test"),
                ("Brazil's jobless rate hits lowest since 2014 for a quarter through June", "https://bit.ly/test"),
                ("Auction sizes set to increase at US Treasury refunding", "https://bit.ly/test"),
                ("EU stress test shows three banks falling short", "https://bit.ly/test")]
news += add_news_source("Reuters - Macro Matters", reuters_news)

ft_news = [("SEC asked Coinbase to halt trading in everything except bitcoin, CEO says", "https://bit.ly/test"),
           ("Rouble-Tether crypto trading surged as Wagner rebellion erupted", "https://bit.ly/test"),
           ("Oil majors to face energy transition scrutiny as war profit boost fades", "https://bit.ly/test"),
           ("Can Starling Bank make tech its business?", "https://bit.ly/test"),
           ("Rishi Sunak to expand North Sea drilling and carbon capture", "https://bit.ly/test")]
news += add_news_source("Financial Times - Most Read: Markets", ft_news)

In [ ]:
table_news = Table(news, colWidths=[7.1*inch])
table_news.setStyle(table_news_style)

In [ ]:
story.append(table_overview)
story.append(Spacer(1, 0.3*inch))
story.append(table_graph)
story.append(Spacer(1, 0.3*inch))
story.append(table_mover)
story.append(Spacer(1, 0.3*inch))
story.append(table_events)
story.append(PageBreak())
story.append(header_table)
story.append(Spacer(1, 0.3*inch))
story.append(table_news)
doc.build(story)

In [ ]:
image_files = ["nasdaq.png", "sp500.png", "sectors.png"]
for image_file in image_files:
    image_path = os.path.join(assets_folder, image_file)
    if os.path.exists(image_path):
        os.remove(image_path)